In [1]:
from linear_models_functions import *

In [66]:
features = ['Score','White Percent', 'Black Percent', 'Hispanic Percent', 'High School Diploma or Higher', "Bachelor's Degree or higher", 'With an Advanced Degree', 'Percent Pop of US', 'Density', 'Average Income', 'Peak Unemployment', 'COVID Cases', 'COVID Deaths', 'Population', 'COVID Cases (% of Population)']

features_a = ['Score', 'Density', 'White Percent', 'Hispanic Percent', "Bachelor's Degree or higher", 'Average Income', 'COVID Cases (% of Population)', 'Peak Unemployment']

dates = ['2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06']
dates2 = ['2020-03', '2020-04', '2020-05', '2020-06']

#run_models('ALCOHOL', dates2, plot=True)
#summarize_models_dem('DRINKING', waves, wave=True)
#summarize_LASSO('ALCOHOL', dates2, features)
summarize_models('LIQUOR', ['2020-06'], features_a)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.328
Model:                            OLS   Adj. R-squared:                  0.200
Method:                 Least Squares   F-statistic:                     2.566
Date:                Wed, 14 Jul 2021   Prob (F-statistic):             0.0224
Time:                        16:04:39   Log-Likelihood:                 392.29
No. Observations:                  51   AIC:                            -766.6
Df Residuals:                      42   BIC:                            -749.2
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const         

/home/johnattan/.local/share/virtualenvs/CHIP-GhDHLp-X/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
